# CIFAR 10 dataset - Numpy

In [1]:
%load_ext autotime

time: 140 µs (started: 2021-04-16 16:32:18 +02:00)


In [2]:
import os
import platform
import pickle
import numpy as np

import torch  

time: 315 ms (started: 2021-04-16 16:32:18 +02:00)


In [3]:
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

time: 262 µs (started: 2021-04-16 16:32:19 +02:00)


In [4]:
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000,3,32,32)
        Y = np.array(Y)
        return X, Y

time: 233 µs (started: 2021-04-16 16:32:19 +02:00)


In [5]:
def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte


time: 276 µs (started: 2021-04-16 16:32:19 +02:00)


In [6]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    # Load the raw CIFAR-10 data
    cifar10_dir = 'CIFAR-10-DS/cifar-10-batches-py/'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    x_train = X_train.astype('float32')
    x_test = X_test.astype('float32')

    x_train /= 255
    x_test /= 255

    #return X_train, y_train, X_test, y_test
    #return x_train, y_train, X_val, y_val, x_test, y_test 

    return x_train, y_train, x_test, y_test 

time: 296 µs (started: 2021-04-16 16:32:19 +02:00)


In [7]:
# Invoke the above function to get our data.
x_train, y_train, x_test, y_test = get_CIFAR10_data()

#print('x_train ', type(x_train), x_train.shape)
#print('y_train', y_train.shape)

#print('x_test', x_test.shape)
#print('y_test', y_test.shape)



time: 547 ms (started: 2021-04-16 16:32:19 +02:00)


### Create batches

In [8]:
def createBatches(data,batch_size_required):
    batch_size = int(len(data)/batch_size_required)
    res=[]
    #print('Data ',len(data))
    for i in range (batch_size):
        batched_data = data[i*batch_size_required:i*batch_size_required+batch_size_required]
        res.append(batched_data)
        #print('Check here', len(batched_data))
    res = np.asarray(res)
    #print('Res ',len(res))
    return res

time: 253 µs (started: 2021-04-16 16:32:19 +02:00)


In [9]:
batch_size = 64
x_train = createBatches(x_train,64)
y_train = createBatches(y_train,64)

x_test = createBatches(x_test,64)
y_test = createBatches(y_test,64)

time: 191 ms (started: 2021-04-16 16:32:19 +02:00)


## Convert npArray to tensor

In [10]:
x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)

x_test_tensor = torch.as_tensor(x_test)
y_test_tensor = torch.as_tensor(y_test)

#print('X tensor ', type(x_test_tensor))

time: 422 µs (started: 2021-04-16 16:32:19 +02:00)


### Setting up the layers of Convolutional neural network

In [11]:
in_size = 3                 # number of channel in the input image
 
hid1_size = 64              # no of output channel from first CNN layer

hid2_size = 128             # no of output channel from second CNN layer

hid3_size = 256             # no of output channel from third CNN layer

hid4_size = 512             # no of output channel from forth CNN layer

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

out_size = len(classes)     # no of categories in the dataset

k_conv_size = 3             # 3x3 convolutional kernel

conv_stride = 1             # conv stride 1

conv_pad = 1                # conv padding 1

maxpool_kernel = 2          # maxpool layer kernel size 2 x 2

maxpool_stride = 2          # maxpool layer stride 2

max_epoch = 1               # Maximun numbe of epochs

time: 291 µs (started: 2021-04-16 16:32:19 +02:00)


In [12]:
import torch.nn as nn

class VGG_11(nn.Module):
    
    def __init__(self):
        super(VGG_11,self).__init__()
        
        self.convLayer = nn.Sequential(
            nn.Conv2d(in_size, hid1_size, k_conv_size, stride=conv_stride, padding=conv_pad),    # conv layer
            nn.BatchNorm2d(hid1_size),
            #nn.LocalResponseNorm(64),
            nn.ReLU(),                              # Activation layer
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid1_size,hid2_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid2_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid2_size,hid3_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid3_size),
            nn.ReLU(),
            
            nn.Conv2d(hid3_size,hid3_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid3_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid3_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride), 
            
            #nn.Flatten(0,-1),
            
            #nn.Linear(512*bth_size, out_size)
            
        )
        
     
        self.fullyConnLayer = nn.Sequential(
            nn.Linear(512, out_size),
            #nn.Softmax(dim=1)
        )
            
        
    def forward(self,x):
            out = self.convLayer(x)

            out = out.reshape(out.size(0), -1)
            
            out = self.fullyConnLayer(out)
            
            return out 
        
vgg_11 = VGG_11()

time: 34.9 ms (started: 2021-04-16 16:32:19 +02:00)


## Define a Loss function and optimizer

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(vgg_11.parameters())

time: 581 µs (started: 2021-04-16 16:32:20 +02:00)


## Train the network

In [14]:
for epoch in range(max_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(x_train_tensor, 0):
        #inputs, labels = data
        
        #images = data
        #print(data.shape)
        inputs = data
        labels = y_train_tensor[i]
        optimizer.zero_grad()
        outputs = vgg_11(inputs)
        
        #print(type(outputs))
        #print(type(labels))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # print statistics        
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.235
[1,   400] loss: 1.690
[1,   600] loss: 1.457
Finished Training
time: 13min 11s (started: 2021-04-16 16:32:20 +02:00)


### Accuracy on whole test-dataset

In [16]:
correct = 0
total = 0
with torch.no_grad():
    for i,data in enumerate(x_test_tensor, 0):
        #images, labels = data
        #print('test ', len(data))
        images = data
        labels = y_test_tensor[i]

        outputs = vgg_11(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 58 %
time: 23.7 s (started: 2021-04-16 16:47:59 +02:00)


### Which classes perform well and which classes did not?

In [17]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for i, data in enumerate(x_test_tensor, 0):
        #images, labels = data
        images = data
        labels = y_test_tensor[i]

        outputs = vgg_11(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 78 %
Accuracy of   car : 80 %
Accuracy of  bird : 13 %
Accuracy of   cat : 53 %
Accuracy of  deer : 38 %
Accuracy of   dog : 22 %
Accuracy of  frog : 73 %
Accuracy of horse : 57 %
Accuracy of  ship : 75 %
Accuracy of truck : 65 %
time: 23.9 s (started: 2021-04-16 16:48:27 +02:00)


# END